In [1]:
import nibabel as nib
import pandas as pd
import numpy as np

def extract_values_from_coordinates(file_paths, coordinates, example_img_path):
    """
    Extract values from NIfTI files based on MNI coordinates and store them in a Pandas DataFrame.
    
    Parameters:
        file_paths (list): List of paths to the NIfTI files.
        coordinates (list): List of dictionaries containing MNI coordinates in the form {'x': x_val, 'y': y_val, 'z': z_val}.
        example_img_path (str): Path to an example NIfTI image to get the affine matrix from.
        
    Returns:
        pd.DataFrame: DataFrame with NIfTI file paths as rows and coordinates as columns.
    """
    # Load the example NIfTI file to get the affine matrix
    example_img = nib.load(example_img_path)
    affine = example_img.affine
    inv_affine = np.linalg.inv(affine)
    
    # Initialize an empty DataFrame
    df = pd.DataFrame(index=file_paths)
    
    for coord in coordinates:
        col_name = f"x{coord['x']}_y{coord['y']}_z{coord['z']}"
        values = []
        
        for file_path in file_paths:
            try:
                img = nib.load(file_path)
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                continue
            
            data = img.get_fdata()
            
            mni_coord = np.array([coord['x'], coord['y'], coord['z'], 1])
            voxel_coord = np.dot(inv_affine, mni_coord)
            i, j, k, _ = np.round(voxel_coord).astype(int)
            
            try:
                value = data[i, j, k]
            except IndexError:
                print(f"Invalid coordinate {coord} for file {file_path}")
                value = np.nan
            
            values.append(value)
        
        df[col_name] = values
    
    return df
